# AutoML: Train "the best" classifier model for the UCI Bank Marketing dataset.

**Requirements** - In order to benefit from this tutorial, you will need:
- A basic understanding of Machine Learning
- An Azure account with an active subscription. [Create an account for free](https://azure.microsoft.com/free/?WT.mc_id=A261C142F)
- An Azure ML workspace. [Check this notebook for creating a workspace](/sdk/resources/workspace/workspace.ipynb) 
- A Compute Cluster. [Check this notebook to create a compute cluster](/sdk/resources/compute/compute.ipynb)
- A python environment
- Installed Azure Machine Learning Python SDK v2 - [install instructions](/sdk/README.md#getting-started)
- The latest MLFlow packages:

    pip install azureml-mlflow

    pip install mlflow

**Learning Objectives** - By the end of this tutorial, you should be able to:
- Connect to your AML workspace from the Python SDK
- Create an `AutoML classification Job` with the 'classification()' factory-fuction.
- Submit and run the AutoML classification job
- Obtaing the model and score predictions with it

**Motivations** - This notebook explains how to setup and run an AutoML classification job. This is one of the nine ML-tasks supported by AutoML. Other ML-tasks are 'regression', 'time-series forecasting', 'image classification', 'image object detection', 'nlp text classification', etc.

In this example we use the UCI Bank Marketing dataset to showcase how you can use AutoML for a classification problem. The classification goal is to predict if the client will subscribe to a term deposit with the bank.

# 1. Connect to Azure Machine Learning Workspace

The [workspace](https://docs.microsoft.com/en-us/azure/machine-learning/concept-workspace) is the top-level resource for Azure Machine Learning, providing a centralized place to work with all the artifacts you create when you use Azure Machine Learning. In this section we will connect to the workspace in which the job will be run.

## 1.1. Import the required libraries

In [54]:
# Import required libraries
from azure.identity import DefaultAzureCredential
from azure.identity import InteractiveBrowserCredential
from azure.ml import MLClient

from azure.ml._constants import AssetTypes
from azure.ml import automl
from azure.ml.entities import JobInput

import mlflow

from pprint import pprint

## 1.2. Workspace details

In [55]:
#Enter details of your AML workspace

# CDLTLL
region = 'centraluseuap' #'<REGION>' ## example: westus
subscription_id = '102a16c3-37d3-48a8-9237-4c9b1e8e80e0' #'<SUBSCRIPTION_ID>'
resource_group = 'automlpmdemo' # '<RESOURCE_GROUP>'
workspace = 'cesardl-automl-centraluseuap-ws' # '<AML_WORKSPACE_NAME>'

# SAGAR
# region = 'centraluseuap' #'<REGION>' ## example: westus
# subscription_id = "381b38e9-9840-4719-a5a0-61d9585e1e91" #'<SUBSCRIPTION_ID>'
# resource_group = "sasum_centraluseuap_rg" # '<RESOURCE_GROUP>'
# workspace = "sasum-centraluseuap-ws" # '<AML_WORKSPACE_NAME>'

## 1.2 Obtain the tracking URI for MLFlow

In [56]:
# This URI can also be found with a CLI command:
# /> az ml workspace show --query mlflow_tracking_uri

## Construct AzureML MLFLOW TRACKING URI
def get_azureml_mlflow_tracking_uri(region, subscription_id, resource_group, workspace):
    return "azureml://{}.api.azureml.ms/mlflow/v1.0/subscriptions/{}/resourceGroups/{}/providers/Microsoft.MachineLearningServices/workspaces/{}".format(region, subscription_id, resource_group, workspace)

MLFLOW_TRACKING_URI = get_azureml_mlflow_tracking_uri(region, subscription_id, resource_group, workspace)

## Make sure the MLflow URI looks something like this: 
## azureml://<REGION>.api.azureml.ms/mlflow/v1.0/subscriptions/<SUBSCRIPTION_ID>/resourceGroups/<RESOURCE_GROUP>/providers/Microsoft.MachineLearningServices/workspaces/<AML_WORKSPACE_NAME>

print("MLFlow Tracking URI:", MLFLOW_TRACKING_URI)

MLFlow Tracking URI: azureml://centraluseuap.api.azureml.ms/mlflow/v1.0/subscriptions/102a16c3-37d3-48a8-9237-4c9b1e8e80e0/resourceGroups/automlpmdemo/providers/Microsoft.MachineLearningServices/workspaces/cesardl-automl-centraluseuap-ws


In [57]:
# Set the MLFLOW TRACKING URI

mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)

print("\nCurrent tracking uri: {}".format(mlflow.get_tracking_uri()))


Current tracking uri: azureml://centraluseuap.api.azureml.ms/mlflow/v1.0/subscriptions/102a16c3-37d3-48a8-9237-4c9b1e8e80e0/resourceGroups/automlpmdemo/providers/Microsoft.MachineLearningServices/workspaces/cesardl-automl-centraluseuap-ws


## 1.3. Configure workspace details and get a handle to the workspace

To connect to a workspace, we need identifier parameters - a subscription, resource group and workspace name. We will use these details in the `MLClient` from `azure.ml` to get a handle to the required Azure Machine Learning workspace. We use the default [interactive authentication](https://docs.microsoft.com/en-us/python/api/azure-identity/azure.identity.interactivebrowsercredential?view=azure-python) for this tutorial. More advanced connection methods can be found [here](https://docs.microsoft.com/en-us/python/api/azure-identity/azure.identity?view=azure-python).

In [58]:
#get a handle to the workspace
credential = InteractiveBrowserCredential() # DefaultAzureCredential()
#credential = DefaultAzureCredential()
ml_client = MLClient(credential, subscription_id, resource_group, workspace)

## 1.4 Initialize MLFlow Client
The models and artifacts that are produced by AutoML can be accessed via the MLFlow interface. 
Initialize the MLFlow client here, and set the backend as Azure ML, via. the MLFlow Client.

*IMPORTANT*, remember you need to have installed the latest MLFlow packages with:

    pip install azureml-mlflow

    pip install mlflow

In [59]:
from mlflow.tracking.client import MlflowClient

# Initialize MLFlow client
mlflow_client = MlflowClient()

In [60]:
# Set the experiment name in MLFlow
# Experiment Name is used in MLFlow and AutoML Job, later.
exp_name = "dpv2-sdk-classifier-experiment"
mlflow.set_experiment(exp_name)

2022/03/30 18:54:59 INFO mlflow.tracking.fluent: Experiment with name 'dpv2-sdk-classifier-experiment' does not exist. Creating a new experiment.


<Experiment: artifact_location='', experiment_id='3ba5733e-d535-4eec-93bf-3da829f5be00', lifecycle_stage='active', name='dpv2-sdk-classifier-experiment', tags={}>

# 2. Configure and run the AutoML classification job
In this section we will configure and run the AutoML classification job.

## 2.1 Configure the job through the classification() factory function

### classification() parameters:

The `classification()` factory function allows user to configure AutoML for the classification task for the most common scenarios with the following properties.

- `target_column_name` - The name of the column to target for predictions. It must always be specified. This parameter is applicable to 'training_data', 'validation_data' and 'test_data'.
- `primary_metric` - The metric that AutoML will optimize for Classification model selection.
- `training_data` - The data to be used for training. It should contain both training feature columns and a target column. Optionally, this data can be split for segregating a validation or test dataset. 
You can use a registered MLTable in the workspace using the format '<mltable_name>:<version>' OR you can use a local file or folder as a MLTable. For e.g JobInput(mltable='my_mltable:1') OR JobInput(mltable=MLTable(local_path="./data"))
The parameter 'training_data' must always be provided.
- `compute` - The compute on which the AutoML job will run. In this example we are using a compute called 'cpu-cluster' present in the workspace. You can replace it any other compute in the workspace. 
- `name` - The name of the Job/Run. This is an optional property. If not specified, a random name will be generated.
- `experiment_name` - The name of the Experiment. An Experiment is like a folder with multiple runs in Azure ML Workspace that should be related to the same logical machine learning experiment.

### set_limits() parameters:
This is an optional configuration method to configure limits parameters such as timeouts.     
    
- timeout_minutes - Maximum amount of time in minutes that the whole AutoML job can take before the job terminates. This timeout includes setup, featurization and training runs but does not include the ensembling and model explainability runs at the end of the process since those actions need to happen once all the trials (children jobs) are done. If not specified, the default job's total timeout is 6 days (8,640 minutes). To specify a timeout less than or equal to 1 hour (60 minutes), make sure your dataset's size is not greater than 10,000,000 (rows times column) or an error results.

- trial_timeout_minutes - Maximum time in minutes that each trial (child job) can run for before it terminates. If not specified, a value of 1 month or 43200 minutes is used.
    
- max_trials - The maximum number of trials/runs each with a different combination of algorithm and hyperparameters to try during an AutoML job. If not specified, the default is 1000 trials. If using 'enable_early_termination' the number of trials used can be smaller.
    
- max_concurrent_trials - Represents the maximum number of trials (children jobs) that would be executed in parallel. It's a good practice to match this number with the number of nodes your cluster.
    
- enable_early_termination - Whether to enable early termination if the score is not improving in the short term. 
    

In [61]:
# Create MLTables for training dataset

my_training_data_input = JobInput(type=AssetTypes.MLTABLE, path="./mltable-folder")

# Remote MLTable definition
# my_training_data_input  = JobInput(type=AssetTypes.MLTABLE, path="azureml://datastores/workspaceblobstore/paths/Classification/Train")


In [62]:
# Create the AutoML classification job with the related factory-function.

classification_job = automl.classification(
                        compute = "cpu-cluster",
                        # name="dpv2-classifier-job-02",
                        experiment_name = exp_name,
                        training_data = my_training_data_input,
                        target_column_name = "y",
                        primary_metric = "accuracy",
                        n_cross_validations=5,
                        enable_model_explainability=True,
                        tags={"my_custom_tag": "My custom value"},

                        # These are temporal properties needed in Private Preview
                        properties={
                            "_automl_internal_enable_mltable_quick_profile": True,
                            "_automl_internal_label": "latest",
                            "_automl_internal_save_mlflow": True
                        }
)

# Limits are all optional
classification_job.set_limits(timeout = 600, #timeout_minutes
                              trial_timeout = 20, #trial_timeout_minutes
                              max_trials = 5, 
                              max_concurrent_trials = 4,
                              # max_cores_per_trial: -1,
                              enable_early_termination=True)

# Training properties are optional
classification_job.set_training(blocked_models=["LogisticRegression"],
                                enable_onnx_compatible_models=True)

# Featurization properties are optional
# classification_job.set_featurization(# drop_columns=["not_needed_column"], # Optional
#                                      # enable_dnn_featurization=True         # Enable if there are text columns
#                                     )


## 2.2 Run the CommandJob
Using the `MLClient` created earlier, we will now run this CommandJob in the workspace.

In [63]:
# Submit the AutoML job
returned_job = ml_client.jobs.create_or_update(classification_job)  # submit the job to the backend


In [64]:
print(f"Created job: ")
returned_job

Created job: 


ClassificationJob({'log_verbosity': 'Info', 'task_type': <TaskType.CLASSIFICATION: 'Classification'>, 'environment_id': None, 'environment_variables': None, 'outputs': {}, 'display_name': '182d7e7d-412d-4a65-8795-92f881660837', 'type': 'automl', 'status': 'NotStarted', 'log_files': None, 'name': '182d7e7d-412d-4a65-8795-92f881660837', 'description': None, 'tags': {'my_custom_tag': 'My custom value'}, 'properties': {'_automl_internal_enable_mltable_quick_profile': 'True', '_automl_internal_label': 'latest', '_automl_internal_save_mlflow': 'True', 'mlflow.source.git.repoURL': 'git@github.com:Azure/azureml-examples.git', 'mlflow.source.git.branch': 'automl-preview', 'mlflow.source.git.commit': 'ba86c6051764c60c70bb858f38eb4cb76dc59f1e', 'azureml.git.dirty': 'True'}, 'id': '/subscriptions/102a16c3-37d3-48a8-9237-4c9b1e8e80e0/resourceGroups/automlpmdemo/providers/Microsoft.MachineLearningServices/workspaces/cesardl-automl-centraluseuap-ws/jobs/182d7e7d-412d-4a65-8795-92f881660837', 'base_pa

In [65]:
# Get a URL for the status of the job
returned_job.services["Studio"].endpoint

'https://ml.azure.com/runs/182d7e7d-412d-4a65-8795-92f881660837?wsid=/subscriptions/102a16c3-37d3-48a8-9237-4c9b1e8e80e0/resourcegroups/automlpmdemo/workspaces/cesardl-automl-centraluseuap-ws&tid=72f988bf-86f1-41af-91ab-2d7cd011db47'

In [50]:
print(returned_job.name)

995b064e-5c51-4592-b53a-14314d1de067


# Retrieve the Best Trial (Best Model's trial/run)
Or any trial based on a custom criteria. Use the MLFLowClient to access the results (such as Models, Artifacts, Metrics) of a previously completed AutoML Trial.

### Get the parent run

In [41]:
job_name = returned_job.name

# Example if providing an specific Job name/ID
# job_name = "c15b6d6f-969b-40b4-b3d7-2b2ef9d26226"

# mlflow_client was initialized at the begining of the notebook

# Get the parent run
mlflow_parent_run = mlflow_client.get_run(job_name)

# Print it
# mlflow_parent_run

In [52]:
mlflow_parent_run.data.tags

# If 'automl_best_child_run_id' is not there, wait until it's there.
# Do not run the next cell until 'automl_best_child_run_id' tag is there.

{'my_custom_tag': 'My custom value',
 'model_explain_run': 'best_run',
 'pipeline_id': '',
 'score': '',
 'predicted_cost': '',
 'fit_time': '',
 'training_percent': '',
 'iteration': '',
 'run_preprocessor': '',
 'run_algorithm': '',
 'automl_best_child_run_id': '995b064e-5c51-4592-b53a-14314d1de067_4',
 'model_explain_best_run_child_id': '995b064e-5c51-4592-b53a-14314d1de067_4',
 'mlflow.rootRunId': '995b064e-5c51-4592-b53a-14314d1de067',
 'mlflow.runName': '995b064e-5c51-4592-b53a-14314d1de067'}

### Get the best model's child run

**IMPORTANT:** This cell might fail if the tags data are still not available in the parent run.
That data is created with 'eventual consistency' so sometimes might not be there instantly.

TO DO: We might need a synchronization mechanism to "wait for eventual consistency", "wait until the 'automl_best_child_run_id' tag is there...".

In [40]:
# Get the best model's child run

best_child_run_id = mlflow_parent_run.data.tags["automl_best_child_run_id"]
print("Found best child run id: ", best_child_run_id)

best_run = mlflow_client.get_run(best_child_run_id)

# best_run

Found best child run id:  995b064e-5c51-4592-b53a-14314d1de067_4


## Get best model run's metrics

In [36]:
import pandas as pd

pd.DataFrame(best_run.data.metrics, index=[0]).T

,0
precision_score_macro,0.799371
AUC_macro,0.949015
average_precision_score_weighted,0.956556
log_loss,0.194514
balanced_accuracy,0.736779
accuracy,0.915266
precision_score_weighted,0.908142
weighted_accuracy,0.959601
precision_score_micro,0.915266
recall_score_micro,0.915266


## Download the best model locally

In [37]:
# Create local folder
local_dir = "./artifact_downloads"
if not os.path.exists(local_dir):
    os.mkdir(local_dir)

In [38]:
# Download run's artifacts/outputs
local_path = mlflow_client.download_artifacts(best_run.info.run_id, "outputs", local_dir)
print("Artifacts downloaded in: {}".format(local_path))
print("Artifacts: {}".format(os.listdir(local_path)))



Artifacts downloaded in: C:\Users\CESARDL.CESARDLSB2-MSFT\GitAMLRepos\azureml-examples-automl-preview-branch\sdk\jobs\automl-standalone-jobs\automl-classification-task-bankmarketing\artifact_downloads\outputs
Artifacts: ['conda_env_v_1_0_0.yml', 'engineered_feature_names.json', 'env_dependencies.json', 'featurization_summary.json', 'internal_cross_validated_models.pkl', 'model.pkl', 'pipeline_graph.json', 'run_id.txt', 'scoring_file_v_1_0_0.py', 'scoring_file_v_2_0_0.py']


In [53]:
%%sh
cat ./artifact_downloads/outputs/MLmodel

Couldn't find program: 'sh'


# Next Steps
You can see further examples of other AutoML tasks such as Image-Classification, Image-Object-Detection, NLP-Text-Classification, Time-Series-Forcasting, etc.